# explorer
fill a playlist with all tracks or just top tracks from every artist specified from:
- a playlist
- artists user is following
- a single artist
- saved songs
- a genre

## references
- https://spotipy.readthedocs.io/en/latest/
- https://developer.spotify.com/documentation/general/guides/scopes/

In [1]:
# setup 
import spotipy
import spotipy.util as util
import os
import sys
import pprint
import string
import traceback

In [3]:
## API info
SPOTIPY_CLIENT_ID     = 'client-id'
SPOTIPY_CLIENT_SECRET = 'client-secret'
SPOTIPY_REDIRECT_URI  = 'redirect-url'

# set environment variables - this may or may not be necessary
os.environ['SPOTIPY_CLIENT_ID']     = SPOTIPY_CLIENT_ID
os.environ['SPOTIPY_CLIENT_SECRET'] = SPOTIPY_CLIENT_SECRET
os.environ['SPOTIPY_REDIRECT_URI']  = SPOTIPY_REDIRECT_URI
my_username = "my-username"

# set permissions scope
my_scope = 'user-library-read user-follow-read playlist-read-collaborative playlist-read-private playlist-modify-private playlist-modify-public'

# set up request token
token = util.prompt_for_user_token(
        username=my_username,
        scope=my_scope,
        client_id=SPOTIPY_CLIENT_ID,
        client_secret=SPOTIPY_CLIENT_SECRET,
        redirect_uri=SPOTIPY_REDIRECT_URI)

In [4]:
# get source
sources = ['playlist', 'artists', 'artist', 'saved', 'genre']
source_type = input("Where do you want to get artists from?\n(1) playlist\n(2) followed artists\n(3) artist\n(4) saved songs\n(5)genre\n")
print(sources[int(source_type) - 1])

source = sources[int(source_type) - 1] 

if source == 'playlist':
    artist_name = input('Which playlist?\n')
elif source == 'artist':
    artist_name = input('Which artist?\n')
elif source == 'genre':
    genre_name = input('Which genre?\n')
    
depths = ['all', 'top']
depth = input("How many tracks do you want from each?\n(1) All\n(2) Top 5\n")
print(depths[int(depth) - 1])

Where do you want to get artists from?
(1) playlist
(2) followed artists
(3) artist
(4) saved songs
(5)genre
1
playlist
Which playlist?
n_p
How many tracks do you want from each?
(1) All
(2) Top 5
1
all


In [5]:
# helper functions
def get_followed_artists(spot):
    '''
    Get all the artists the user follows
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all artists the user follows
    '''
    results = spot.current_user_followed_artists()
    all_artists = results['items']
    while results['next']:
        results = spot.next(results)
        all_artists.extend(results['items'])
    return all_artists

def get_playlists(spot):
    '''
    Get all the user's playlists
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all user's playlists
    '''
    results = spot.current_user_playlists()
    all_playlists = results['items']
    while results['next']:
        results = spot.next(results)
        all_playlists.extend(results['items'])
    return all_playlists

def get_playlist_artists(spot, user_id, playlist_id):
    '''
    Get all the artists that appear on a playlist
    
    Arguments:
        spot        - spotipy object
        user_id     - user id
        playlist_id - playlist id 
        
    Returns:
        list - all artists that appear
    '''
    results = spot.user_playlist_tracks(user_id, playlist_id)
    all_tracks = results['items']
    while results['next']:
        results = spot.next(results)
        all_tracks.extend(results['items'])
    
    all_artists = []
    for i in range(len(all_tracks)):
        for j in range(len(all_tracks[i]['track']['artists'])):
            name = all_tracks[i]['track']['artists'][j]['name']
            # getting weird occurence of an empty artist name, so filter out
            if name != '':
                # is a set, so only new artists will be added
                all_artists.add(name)

def get_artist_id(spot, artist_name):
    '''
    '''
    return None

def get_artist_tracks(spot, artist_id):
    '''
    '''
    return None

def get_saved_tracks(spot):
    '''
    Get all tracks the user has saved
    
    Arguments:
        spot - spotipy object
        
    Returns:
        list - all saved songs
    '''
    results = spot.current_user_saved_tracks()
    all_saved_tracks = results['items']
    while results['next']:
        results = spot.next(results)
        all_saved_tracks.extend(results['items'])
    return all_saved_tracks

def get_genre_artists(sp, genre_name):
    '''
    '''
    return None

In [8]:
# user setup
user_id = sp.me()

if source == 'playlist':
    # get playlist artists
    playlist_name = 'idk'
    all_playlists = get_playlists(sp)

    playlist_artists = []
    for playlist in all_playlists:
        if playlist['name'] == playlist_name:
            playlist_artists = get_playlist_artists(sp, user_id, playlist['id'])

elif source == 'artists':
    followed_artists = get_followed_artists(sp)
    
elif source == 'artist':
    artist_tracks = get_artist_tracks(sp, artist_name)
    
elif source == 'saved':
    saved_tracks = get_saved_tracks(sp)
    
elif source == 'genre':
    genre_artists = get_genre_artists(sp, genre_name)

NameError: name 'sp' is not defined